# Movie Classifier

*The main focus of this notebook is to develop machine learning models that can accurately classify the ***"categories"*** of movies or series. These models rely on a set of features that have proven to be valuable during the preprocessing stage. The project involves several significant phases, namely ***data analysis***, ***preprocessing***, ***model training***, ***optimization***, and ***prediction***. The training process will utilize ***decision tree*** and ***random forest*** algorithms as the primary methods.*

# *Importing Libraries*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn

# *Imprting Dataset*

Our analysis will be based on a dataset comprising Netflix and Amazon Series/Movies. The dataset includes various fields and attributes that provide relevant information for our task, such as:

+ ***Type:*** Referring to the category or genre of the series/movie.
+ ***Title:*** The name or title of the series/movie.
+ ***Cast:*** The actors or actresses involved in the production.
+ ***Country:*** The country or countries where the series/movie was produced.
+ ***Release Year:*** The year when the series/movie was released.
+ ***Listed In:*** The categorization or genres assigned to the series/movie.
+ ***Description:*** A brief description or summary of the series/movie.

Our primary objective is to predict the values associated with the "type" field, utilizing all the other attributes available in the dataset. This prediction will enable us to classify and categorize series/movies based on their attributes.

In [2]:
df = pd.read_csv('dataset.csv')
df

,type,title,cast,country,release_year,listed_in,description
0,Movie,Dick Johnson Is Dead,NaN,United States,2020,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,NaN,NaN,2021,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,2000,"Action-Adventure, Animation, Kids",X-Men: Evolution features the team as teenager...
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,1996,"Comedy, Coming of Age, Kids",A genius tries to fit in as a high school soph...
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,2014,"Action-Adventure, Comedy, Coming of Age",Welcome to Kirby's world! It's rude and sketchy.
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,2016,"Action-Adventure, Comedy, Science Fiction",Ryan discovers his ability to control a giant ...
